In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchmetrics
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import lightning as L

/opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <761A4B43-4CD1-322C-BB16-CEE783FE0A7C> /opt/miniconda3/envs/real-estate-dinov2-test/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [28]:
df = pd.read_pickle("../data/flats_clean.pkl")

In [32]:
class CustomDataset(Dataset):
    def __init__(self, dataset_pickle, images_root, image_transform=None):
        self.df = pd.read_pickle(dataset_pickle)
        self.images_root = images_root
        self.image_transform = image_transform

    def __len__(self):
        return len(self.df)
    
    def get_images(self, root):
        batch = []
        for root, _, files in os.walk(root):
            for file in files:
                if file.endswith(".jpg") and "all" not in file:
                    image_path = os.path.join(root, file)
                    image = Image.open(image_path).convert("RGB")

                    if self.image_transform:
                        image = self.image_transform(image)

                    batch.append(image)
        
        return torch.stack(batch)


    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        offer_nr = row["Numer oferty"]
        images = self.get_images(f"{self.images_root}/{offer_nr}")

        return images

In [35]:
transform = transforms.Compose([
    transforms.Resize((224*2, 224*2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

dataset = CustomDataset(
    dataset_pickle="../data/flats_train.pkl",
    images_root="../images",
    image_transform=transform,
)

images = next(iter(dataset))
print(images.shape)

torch.Size([18, 3, 448, 448])


In [4]:
dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14_reg")

Using cache found in /Users/kacper/.cache/torch/hub/facebookresearch_dinov2_main
/Users/kacper/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Users/kacper/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Users/kacper/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [36]:
output = dinov2_vits14(images)

In [37]:
output.shape

torch.Size([18, 384])